In [16]:
%load_ext autoreload
%autoreload 2
from anapath.params import *
from anapath.Logic.model_diag import initialize_model, compile_model,train_model
from anapath.image.preprocessing import datagenerator_train, datagenerator_val
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers, Sequential, models


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
datagen_train, train_generator = datagenerator_train("diag",(1024,2048))
datagen_val, val_generator = datagenerator_val("diag",(1024,2048))

Found 671 images belonging to 2 classes.
Found 143 images belonging to 2 classes.


In [9]:
train_generator[0][0].shape

(8, 1024, 2048, 1)

In [6]:
model = tf.keras.Sequential([
    # INPUT LAYER
    layers.Input(shape=(2048,1024,1)),

    # CONV/HIDDEN LAYERS
    layers.Conv2D(512, kernel_size=(3, 3), padding='same', kernel_initializer='he_uniform', activation='relu'),
    layers.Conv2D(512, kernel_size=(3, 3), padding='same', kernel_initializer='he_uniform', activation='relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    # CONV/HIDDEN LAYERS
    layers.Conv2D(128, kernel_size=(3, 3), padding='same', kernel_initializer='he_uniform', activation='relu'),
    layers.Conv2D(128, kernel_size=(3, 3), padding='same', kernel_initializer='he_uniform', activation='relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    ### One Fully Connected layer - "Fully Connected" is equivalent to saying "Dense"
    layers.Dense(128, activation='relu',  kernel_initializer='he_uniform'),
    #layers.Dropout(0.2),
    layers.Dense(256, activation='relu',  kernel_initializer='he_uniform'),
    #layers.Dropout(0.2),
    layers.Dense(256, activation='relu',  kernel_initializer='he_uniform'),
    #layers.Dropout(0.2),
    layers.Dense(128, activation='relu',  kernel_initializer='he_uniform'),
    #layers.Dropout(0.2),
    layers.Dense(128, activation='relu',  kernel_initializer='he_uniform'),
    #layers.Dropout(0.2),
    layers.Dense(64, activation='relu',  kernel_initializer='he_uniform'),
    #layers.Dropout(0.2),
    layers.Dense(32, activation='relu',  kernel_initializer='he_uniform'),
    #layers.Dropout(0.2),
    # PREDICITVE LAYER
    layers.Dense(1, activation='sigmoid')
])

In [11]:
model = initialize_model((2048, 1024, 1))

✅ Model initialized


In [12]:
model = compile_model(model, learning_rate = 0.1)

✅ Model compiled


In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 2048, 1024, 512)   5120      
                                                                 
 conv2d_5 (Conv2D)           (None, 2048, 1024, 512)   2359808   
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1024, 512, 512)   0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 1024, 512, 128)    589952    
                                                                 
 conv2d_7 (Conv2D)           (None, 1024, 512, 128)    147584    
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 512, 256, 128)    0         
 2D)                                                  

In [13]:
model, history = train_model(model,train_generator=train_generator,val_generator=val_generator,epochs=1)


Training model...


2025-03-20 08:24:42.930937: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


ValueError: in user code:

    File "/home/gregoire/.pyenv/versions/3.10.6/envs/anapath/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/gregoire/.pyenv/versions/3.10.6/envs/anapath/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/gregoire/.pyenv/versions/3.10.6/envs/anapath/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/gregoire/.pyenv/versions/3.10.6/envs/anapath/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/gregoire/.pyenv/versions/3.10.6/envs/anapath/lib/python3.10/site-packages/keras/engine/training.py", line 1109, in compute_loss
        return self.compi

In [17]:
MODEL_TARGET

'gcs'

In [39]:
from anapath.Logic.registry import load_model

model_tx = load_model("diag")


Load latest model from GCS...
<Blob: anapath_greg-hub-dev, models/20250319-135527.h5, 1742388969299062>
✅ Latest model downloaded from cloud storage


In [40]:
model_tx.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 512, 256, 32)      896       
                                                                 
 conv2d_7 (Conv2D)           (None, 512, 256, 32)      9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 256, 128, 32)     0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 1048576)           0         
                                                                 
 dense_6 (Dense)             (None, 16)                16777232  
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                      

In [ ]:
from PIL import Image
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
test_image = os.path.join(train_tumor_path, "tile_01092_c156_r5_c2.png")
img = Image.open(test_image)
img = img.resize((512, 256))
img = np.array(img)
image_array = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
image_array = np.expand_dims(image_array, axis=0)
datagen = ImageDataGenerator(rescale=1./255)
img_generator = datagen.flow(
    image_array,
    batch_size=1,
    shuffle=False )

img_generator[0].shape
model_tx.predict(img_generator)[0][0]

1/1 [==============================] - 0s 37ms/step


2025-03-20 08:48:22.190505: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


0.5009359

In [1]:
from transformers import AutoImageProcessor, AutoModel

processor = AutoImageProcessor.from_pretrained("owkin/phikon")
model = AutoModel.from_pretrained("owkin/phikon")

2025-03-20 18:07:02.883534: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-20 18:07:02.930196: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-20 18:07:03.151334: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-20 18:07:03.152875: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-20 18:07:04.096617: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
model.summary()

AttributeError: 'ViTModel' object has no attribute 'summary'